## The following are the codes demonstrating the performance of SERM imputation for scRNA-seq data

In [ ]:
# To illustrate the imputation calculation of SERM, the cellular taxonomy dataset analysed in the 
# manuscript will be automatically loaded. However, you can upload your own dataset, and impute the 
# data by SERM. Here the input data is saved as .mat file to reduce the data size (normally .csv file 
# needs more disk space). However, .csv files can also be loaded using python packages such as "pandas"

In [4]:
# Import all the necessary Python packages
import os
from functools import partial
import h5py
import matplotlib.pyplot as plt
import scipy
import scipy.io as sio
from serm import *
from sklearn.decomposition import PCA
import numpy as np
from sklearn.manifold import TSNE
import magic
import random
%matplotlib inline

Load data

In [5]:
# Load the original data (reference data), the downsampled data, and the data labels
# The original data is from the work of Baryawno et al. 
# Reference: Baryawno, Ninib, et al. "A cellular taxonomy 
# of the bone marrow stroma in homeostasis and leukemia." Cell 177.7 (2019): 1915-1932.
# The data from the authors is very large with 23092 cells and 27998 genes. We choose 5% of the
# top highly variable genes (1093 genes) to create a smaller dataset.
# The following two code lines read the reference data described above
a=sio.loadmat('data/data_CellTax_Original.mat')
dataRef=a['dataRef']

# The downsampled data (to 1%) were generated by using the technique described in Miao et al.
# Please see the description of their codes at 
# https://www.bioconductor.org/packages/release/bioc/html/scRecover.html
# We used their "countsSampling" function for the downsampling operation
# Reference: Miao, Z., Li, J., & Zhang, X. (2019). scRecover: Discriminating true and 
# false zeros in single-cell RNA-seq data for imputation. bioRxiv, 665323.
# The following two code lines read the downsampled data
a=sio.loadmat('data/data_CellTax_Downsampled_1perc.mat')
dataSample=a['dataSample']

# Load data labels 
a=sio.loadmat('data/dataLabel_CellTax.mat')
dataLabel=a['GT']

Set parameters of SERM and run it on the downsampled data

In [ ]:
# Selection of the size of the region of interest (ROI)
# We choose ROI size in such a way that the expression matrix
# is divided into 8 (i.e. 4 by 2) ROIs. However, other ROI sizes are also
# fine as SERM is generally very robust against the ROI size. 
ROIsize = [6000,600]
# Selection of percent of overlap between successive ROIs. The default value is 50%, but other 
# values can be chosen
percOL = 0.5
# Run SERM
out_SERM = serm(dataSample,ROIsize,percOL,randomize=False) # If you want to randomize the rows of the data
# before applying SERM, please choose randomize=True

For performance comparison, run MAGIC with default parameters

In [ ]:
magic_operator = magic.MAGIC()
out_MAGIC = magic_operator.fit_transform(dataSample)

Computation of Pearson coefficient for quantitative comparison

In [ ]:
# Here, we compute the Pearson coefficient between the expression values of each gene of the cells from 
# the reference data and imputed data. Thus, for the loaded dataset with 1093 genes, we get 1093 
# coefficients. We then use violin plot to show the  distribution of the coefficients
sz=dataSample.shape
corrVecDownsampled=np.zeros(sz[1])
corrVecSERM=np.zeros(sz[1])
corrVecMAGIC=np.zeros(sz[1])

for i in range(sz[1]):
    
    downSampledGene=dataSample[:,i]
    referenceGene=dataRef[:,i]
    sermGene=out_SERM[:,i]
    
    a = np.corrcoef(downSampledGene, referenceGene)
    corrVecDownsampled[i]=a[1,0]
    
    b= np.corrcoef(sermGene, referenceGene)
    corrVecSERM[i]=b[1,0]
    
    magicGene=out_MAGIC[:,i]
    b= np.corrcoef(magicGene, referenceGene)
    corrVecMAGIC[i]=b[1,0]
    
# Stack all the coefficient values for plotting
corrVec=np.stack((corrVecDownsampled,corrVecMAGIC,corrVecSERM), axis=0)

Create violin plot of the Pearson coefficient values 

In [ ]:
fig, (ax) = plt.subplots(nrows=1, ncols=1, sharey=True)
labels = ['Downsampled', 'MAGIC', 'SERM']
plt.violinplot(corrVec.T,showmeans=True)
plt.title('Comaprison of different imputation techniques')
set_axis_style(ax, labels)
ax.set_ylabel('Pearson coefficient')

t-SNE visualization of the reference, the downsampled, and the imputed data

In [ ]:
tsne_Original = TSNE(n_components=2, learning_rate='auto',init='random',random_state=1).fit_transform(dataRef)
plt.figure(1)
plt.scatter(tsne_Original[:,0],tsne_Original[:,1], c=dataLabel.flatten(),s=1)
plt.xlabel("tSNE1")
plt.ylabel("tSNE2")
plt.title('t-SNE visualization of reference data')

tsne_Downsampled = TSNE(n_components=2, learning_rate='auto',init='random',random_state=1).fit_transform(dataSample)
plt.figure(2)
plt.scatter(tsne_Downsampled[:,0],tsne_Downsampled[:,1], c=dataLabel.flatten(),s=1)
plt.xlabel("tSNE1")
plt.ylabel("tSNE2")
plt.title('t-SNE visualization of downsampled data')
plt.ylim(-30, 30)

tsne_SERM = TSNE(n_components=2, learning_rate='auto',init='random',random_state=1).fit_transform(out_SERM)
plt.figure(3)
plt.scatter(tsne_SERM[:,0],tsne_SERM[:,1], c=dataLabel.flatten(),s=1)
plt.xlabel("tSNE1")
plt.ylabel("tSNE2")
plt.title('t-SNE visualization of SERM-imputed data')

tsne_MAGIC = TSNE(n_components=2, learning_rate='auto',init='random',random_state=1).fit_transform(out_MAGIC)
plt.figure(4)
plt.scatter(tsne_MAGIC[:,0],tsne_MAGIC[:,1], c=dataLabel.flatten(),s=1)
plt.xlabel("tSNE1")
plt.ylabel("tSNE2")
plt.title('t-SNE visualization of MAGIC-imputed data')